In [1]:
import pandas as pd
import numpy as np

rxn = "borylation"
#rxn = "dioxirane"
if rxn == "dioxirane":
    df     = pd.read_csv('../active_learning/regression/perf_clean_run.csv')
    max_   = 135
    rxn_lm = 50
elif rxn == "borylation":
    df     = pd.read_csv('../active_learning/regression/perf_clean_run_borylation_db0-01_a1.csv')
    max_   = 60
    rxn_lm = 22

In [2]:
def get_avg_vs_random(af, df=df, output="mean", max_=135):
    df_ = df[[af, 'random']]
    diff = []
    for i in range(len(df_)):
        if df_[af][i] == max_ and df_['random'][i] == max_: # both are not reaching the accuracy any points
            diff.append(np.nan)
        else:
            diff.append(df_['random'][i] - df_[af][i]) # one of them is reaching the accuracy
    if output == "median":
        return np.median(diff)
    elif output == "mean":
        return np.mean(diff)
    elif output == "diff":
        return diff

In [3]:
df_diff = pd.DataFrame()
for col in df.columns:
    if 'acqf' in col:
        df_diff[col] = get_avg_vs_random(col, df, output="diff", max_=max_)
df_diff["SMILES"] = df["Unnamed: 0"]

In [4]:
df_results_acqf = pd.DataFrame()
non_zero_counts = rxn_lm - df_diff.isna().sum()
df_results_acqf["AF"] = df_diff.columns
df_results_acqf['num_target_acc']            = non_zero_counts.values
df_results_acqf['mean_vs_rd']                = list(df_diff.mean())
df_results_acqf['median_vs_rd']              = list(df_diff.median())

df_results_acqf
#df_results_acqf.to_csv(f"acqf_results_{rxn}.csv")

,AF,num_target_acc,mean_vs_rd,median_vs_rd
0,acqf_1,15,18.200000,22.0
1,acqf_10,17,23.823529,31.0
2,acqf_2,15,15.733333,11.0
3,acqf_2-1,16,19.312500,19.0
4,acqf_3,13,2.230769,2.0
5,acqf_4,14,16.214286,14.0
6,acqf_4-1,15,18.866667,17.0
7,acqf_5,16,20.812500,18.0
8,acqf_6,16,21.125000,20.0
9,acqf_7,15,21.133333,22.0
